In [15]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import os
import re
import pandas as pd
import numpy as np
import sys
sys.path.append('D:\\Git\\cifar10')

from metric import get_metrics
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
cifar_testset = torchvision.datasets.CIFAR10('CIFAR10', train=False, transform=torchvision.transforms.ToTensor(), download=True)
load_testset = DataLoader(cifar_testset, batch_size=64, drop_last=False)


In [16]:
from model.resnet import *
from model.cnn import *
from model.ViT import *
recordf1=[[] for i in range(20)]
recordacc=[[] for i in range(20)]
recordprecision=[[] for i in range(20)]
recordrecall=[[] for i in range(20)]
data_dir = 'data'
#experiment for name
name='resnet'
for filename in os.listdir(data_dir):
    if name in filename:
        model=resnet()
        model=torch.load(os.path.join(data_dir,filename),weights_only=False)
        model.eval()
        f1,acc,precision,recall=get_metrics(model,load_testset,fault_tolerance=2)
        recordf1[int(filename.split('_')[2])]=f1.tolist()
        recordacc[int(filename.split('_')[2])]=acc.tolist()
        recordprecision[int(filename.split('_')[2])]=precision.tolist()
        recordrecall[int(filename.split('_')[2])]=recall.tolist()

In [8]:
from ablation.resnet_dropout import *
model=resnet()
model=torch.load(os.path.join(data_dir,'resnet_cifar10_19_0.6746794581413269.pth'),weights_only=False)
model.eval()
f1,acc,precision,recall=get_metrics(model,load_testset)
print(f1.mean(),acc)

tensor(0.7571, device='cuda:0') tensor(0.7558, device='cuda:0')


In [13]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
pdf1=pd.DataFrame(recordf1,columns=class_names)
pdf1.index.name=None
pdf1.to_csv('result/resnet_recordtop3f1.csv')
pdacc=pd.DataFrame(recordacc,columns=['accuracy'])
pdacc.index.name=None
pdacc.to_csv('result/resnet_recordtop3accuracy.csv')
pdprecision=pd.DataFrame(recordprecision,columns=class_names)
pdprecision.index.name=None
pdprecision.to_csv('result/resnet_recordtop3precision.csv')
pdrecall=pd.DataFrame(recordrecall,columns=class_names)
pdrecall.index.name=None
pdrecall.to_csv('result/resnet_recordtop3recall.csv')
